In [3]:
import os
from pathlib import Path

cwd = Path.cwd().resolve()
if cwd.name == "notebooks_v2":
    os.chdir(cwd.parent)
print("CWD =", Path.cwd().resolve())
assert (Path.cwd() / "src").exists(), "Run this from repo root."

CWD = /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai


In [4]:
%%writefile src/careeragent/langgraph/hitl_flows.py
from __future__ import annotations

import hashlib
from pathlib import Path
from typing import Any, Dict, List, Optional


def _artifacts_root() -> Path:
    return Path("src/careeragent/artifacts").resolve()


def _runs_dir(run_id: str) -> Path:
    d = _artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d


def _write_text(path: Path, text: str) -> str:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(text, encoding="utf-8")
    return str(path)


def _job_key(url: str) -> str:
    # stable short key for filenames
    return hashlib.md5(url.encode("utf-8"), usedforsecurity=False).hexdigest()[:12]  # noqa: S324


def _fallback_resume_md(profile: Dict[str, Any], job: Dict[str, Any]) -> str:
    name = (profile.get("name") or "Candidate").strip()
    contact = profile.get("contact") or {}
    skills = profile.get("skills") or []
    title = job.get("title") or "Target Role"
    matched = job.get("matched_skills") or []
    missing = job.get("missing_skills") or []

    return f"""# {name}
{contact.get('phone','')} | {contact.get('email','')}
{contact.get('linkedin','')}

## Target Role
**{title}**

## Professional Summary
AI/ML + GenAI engineer focused on production-grade delivery (MLOps, CI/CD, evaluation, guardrails). ATS-friendly, high-density keywords, metric-driven bullets.

## Core Skills (ATS)
{", ".join(skills[:30])}

## Matched Skills for This Role
{", ".join(matched[:20]) if matched else "See Core Skills"}

## Skill Gaps
{", ".join(missing[:15]) if missing else "None detected from job text"}

## Experience (ATS bullets)
- Add 4–6 bullets per role with metrics (latency, accuracy, cost, scale, adoption)
- Include tools + scope + measurable impact

## Education
- MSIT (Healthcare Tech) — Clark University (in progress)

## Projects
- CareerAgent-AI: LangGraph orchestration + HITL + explainability + analytics
"""


def _fallback_cover_md(profile: Dict[str, Any], job: Dict[str, Any], country: str) -> str:
    name = (profile.get("name") or "Candidate").strip()
    contact = profile.get("contact") or {}
    title = job.get("title") or "the role"
    url = job.get("url") or ""

    greeting = "Dear Hiring Manager," if country.upper() == "US" else "Dear Hiring Team,"
    return f"""# Cover Letter — {name}

{contact.get('email','')} | {contact.get('phone','')} | {contact.get('linkedin','')}

{greeting}

I’m applying for **{title}**. I build production-grade AI/ML and GenAI systems with strong MLOps discipline (tracking, reproducibility, governance). I focus on measurable outcomes and reliability: evaluation, guardrails, and operational automation.

Highlights:
- End-to-end AI delivery: data → training → evaluation → deployment
- GenAI/LLM apps: RAG, orchestration, tool selection, safety gates
- Cloud + DevOps: CI/CD, containers, monitoring, artifact/version control

Sincerely,  
**{name}**

Job link: {url}
"""


async def approve_ranking_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: HITL approve ranking -> generate drafts (L6) for selected jobs.
    Layer: L6
    """
    run_id = str(state.get("run_id") or "run")
    prefs = state.get("preferences") or {}
    country = str(prefs.get("country") or "US")

    ranking: List[Dict[str, Any]] = state.get("ranking") or []
    approved: List[str] = (state.get("meta") or {}).get("approved_job_urls") or []
    approved = [u.strip() for u in approved if isinstance(u, str) and u.strip()]

    if approved:
        approved_set = set(approved)
        ranking = [j for j in ranking if str(j.get("url") or j.get("job_id") or "") in approved_set]

    if not ranking:
        ranking = (state.get("ranking") or [])[:10]

    # Prefer your real L6 node
    try:
        from careeragent.langgraph.nodes_l6_l9 import l6_draft_node, l6_evaluator_node

        # IMPORTANT: make L6 see only approved jobs
        state["ranking"] = ranking
        state.update(await l6_draft_node(state))      # type: ignore[arg-type]
        state.update(await l6_evaluator_node(state))  # type: ignore[arg-type]
    except Exception:
        # Deterministic fallback drafts (still ATS-friendly)
        prof = state.get("profile") or {}
        run_dir = _runs_dir(run_id)
        artifacts = state.setdefault("artifacts", {})
        drafts = []

        for job in ranking[:10]:
            url = str(job.get("url") or job.get("job_id") or "")
            key = _job_key(url or (job.get("title") or "job"))
            resume_p = run_dir / f"resume_{key}.md"
            cover_p = run_dir / f"cover_{key}.md"

            resume_txt = _fallback_resume_md(prof, job)
            cover_txt = _fallback_cover_md(prof, job, country)

            artifacts[f"resume_{key}"] = {"path": _write_text(resume_p, resume_txt), "content_type": "text/markdown"}
            artifacts[f"cover_{key}"] = {"path": _write_text(cover_p, cover_txt), "content_type": "text/markdown"}

            drafts.append({"job_url": url, "job_title": job.get("title"), "resume_path": artifacts[f"resume_{key}"]["path"], "cover_path": artifacts[f"cover_{key}"]["path"]})

        state["drafts"] = {"drafts": drafts}

    state.setdefault("live_feed", []).append({"layer": "L6", "agent": "HITL", "message": f"Drafts generated for {min(10, len(ranking))} jobs. Review drafts."})
    state["status"] = "needs_human_approval"
    state["pending_action"] = "review_drafts"
    return state


async def approve_drafts_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: HITL approve drafts -> finalize (L7-L9).
    Layer: L7-L9
    """
    try:
        from careeragent.langgraph.nodes_l6_l9 import (
            l7_apply_node, l7_evaluator_node,
            l8_tracker_node, l8_evaluator_node,
            l9_analytics_node,
        )

        state.update(await l7_apply_node(state))       # type: ignore[arg-type]
        state.update(await l7_evaluator_node(state))   # type: ignore[arg-type]
        if state.get("status") == "needs_human_approval":
            return state

        state.update(await l8_tracker_node(state))     # type: ignore[arg-type]
        state.update(await l8_evaluator_node(state))   # type: ignore[arg-type]
        if state.get("status") == "needs_human_approval":
            return state

        state.update(await l9_analytics_node(state))   # type: ignore[arg-type]
    except Exception:
        pass

    state.setdefault("live_feed", []).append({"layer": "L9", "agent": "HITL", "message": "Run completed."})
    state["status"] = "completed"
    state["pending_action"] = None
    return state

Writing src/careeragent/langgraph/hitl_flows.py


In [5]:
%%writefile src/careeragent/api/run_manager_service.py
from __future__ import annotations

import asyncio
import concurrent.futures as cf
import copy
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4

from careeragent.services.db_service import SqliteStateStore
from careeragent.langgraph.runtime_nodes import run_single_layer
from careeragent.langgraph.hitl_flows import approve_ranking_flow, approve_drafts_flow


def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


def artifacts_root() -> Path:
    return Path("src/careeragent/artifacts").resolve()


class RunManagerService:
    """
    Description: Indestructible background runner with hard timeouts + HITL actions.
    Layer: L8
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        state.setdefault("meta", {})
        state["meta"]["heartbeat_utc"] = utc_now()
        self._store.upsert_state(run_id=run_id, status=str(state.get("status", "unknown")), state=state, updated_at_utc=utc_now())

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)
        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            d = float(thresholds["default"])
            thresholds.setdefault("parser", d)
            thresholds.setdefault("discovery", d)
            thresholds.setdefault("match", d)
            thresholds.setdefault("draft", d)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {
                "created_at_utc": utc_now(),
                "heartbeat_utc": utc_now(),
                "last_layer": None,
                "plan_layers": ["L0","L2","L3","L4","L5"],
                "approved_job_urls": [],
            },
            "steps": [],
            "live_feed": [{"layer":"L1","agent":"API","message":"Run created. Starting background pipeline…"}],
            "attempts": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self.save_state(run_id=run_id, state=state)
        return state

    def start_background(self, run_id: str) -> None:
        t = threading.Thread(target=self._bg, args=(run_id,), daemon=True)
        t.start()

    def _call_layer(self, state: Dict[str, Any], layer: str) -> Dict[str, Any]:
        st_copy = copy.deepcopy(state)
        return asyncio.run(run_single_layer(st_copy, layer))

    def _bg(self, run_id: str) -> None:
        state = self.get_state(run_id)
        if not state:
            return

        plan = [("L0", 10), ("L2", 20), ("L3", 45), ("L4", 120), ("L5", 15)]

        for layer, tmo in plan:
            if state.get("status") in ("blocked","needs_human_approval","failed","completed"):
                self.save_state(run_id=run_id, state=state)
                return

            state.setdefault("meta", {})
            state["meta"]["last_layer"] = layer
            state.setdefault("steps", []).append({"layer_id": layer, "status": "running", "started_at_utc": utc_now()})
            state.setdefault("live_feed", []).append({"layer": layer, "agent":"Orchestrator", "message": f"Running {layer}…"})
            self.save_state(run_id=run_id, state=state)

            with cf.ThreadPoolExecutor(max_workers=1) as ex:
                fut = ex.submit(self._call_layer, state, layer)
                try:
                    state = fut.result(timeout=tmo)
                    step_status = "ok"
                except cf.TimeoutError:
                    state["status"] = "needs_human_approval"
                    state["pending_action"] = f"timeout_{layer.lower()}"
                    state.setdefault("live_feed", []).append({"layer": layer, "agent":"TimeoutGuard", "message": f"{layer} timed out after {tmo}s"})
                    step_status = "failed"
                except Exception as e:
                    state["status"] = "failed"
                    state["pending_action"] = f"error_{layer.lower()}"
                    state.setdefault("live_feed", []).append({"layer": layer, "agent":"CrashGuard", "message": f"{layer} crashed: {e}"})
                    step_status = "failed"

            state["steps"][-1]["status"] = step_status
            state["steps"][-1]["finished_at_utc"] = utc_now()
            self.save_state(run_id=run_id, state=state)

            if state.get("pending_action") == "review_ranking":
                state["status"] = "needs_human_approval"
                self.save_state(run_id=run_id, state=state)
                return

        if state.get("status") == "running":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"Orchestrator","message":"Ranking ready for review."})
            self.save_state(run_id=run_id, state=state)

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        state.setdefault("meta", {})
        state["meta"]["last_user_action"] = {"type": action_type, "payload": payload, "at_utc": utc_now()}

        if action_type == "execute_layer":
            layer = str(payload.get("layer","")).upper()
            state = await asyncio.to_thread(lambda: self._call_layer(state, layer))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_ranking":
            selected = payload.get("selected_job_urls") or []
            if isinstance(selected, list):
                state["meta"]["approved_job_urls"] = [str(u).strip() for u in selected if str(u).strip()]
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message": f"Ranking approved. Selected jobs={len(state['meta']['approved_job_urls'])}."})
            state = await asyncio.to_thread(lambda: asyncio.run(approve_ranking_flow(copy.deepcopy(state))))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_ranking":
            reason = str(payload.get("reason","")).strip()
            state.setdefault("meta", {}).setdefault("ranking_reject_reasons", []).append(reason or "no_reason")
            state["status"] = "running"
            state["pending_action"] = None
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message": f"Ranking rejected. Re-running hunt. Reason: {reason[:140]}"} )
            self.save_state(run_id=run_id, state=state)
            self.start_background(run_id)
            return state

        if action_type == "approve_drafts":
            state.setdefault("live_feed", []).append({"layer":"L6","agent":"HITL","message":"Drafts approved. Finalizing…"})
            state = await asyncio.to_thread(lambda: asyncio.run(approve_drafts_flow(copy.deepcopy(state))))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_drafts":
            reason = str(payload.get("reason","")).strip()
            state.setdefault("meta", {}).setdefault("draft_reject_reasons", []).append(reason or "no_reason")
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L6","agent":"HITL","message": f"Drafts rejected. Back to ranking. Reason: {reason[:140]}"} )
            self.save_state(run_id=run_id, state=state)
            return state

        state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state

Overwriting src/careeragent/api/run_manager_service.py


In [6]:
%%writefile app/ui/dashboard.py
from __future__ import annotations

import json
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

import requests
import streamlit as st


def api_base() -> str:
    return os.getenv("API_URL", "http://127.0.0.1:8000").rstrip("/")


def api_get(path: str) -> Dict[str, Any]:
    r = requests.get(api_base() + path, timeout=30)
    r.raise_for_status()
    return r.json()


def api_post(path: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    r = requests.post(api_base() + path, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()


def load_json_artifact(artifact: Optional[Dict[str, Any]]) -> Optional[Any]:
    if not artifact:
        return None
    p = artifact.get("path")
    if not p:
        return None
    fp = Path(p)
    if not fp.exists():
        return None
    return json.loads(fp.read_text(encoding="utf-8"))


def calc_progress(state: Dict[str, Any]) -> int:
    steps = state.get("steps") or []
    done = sum(1 for s in steps if s.get("finished_at_utc"))
    total = max(1, len((state.get("meta") or {}).get("plan_layers") or ["L0","L2","L3","L4","L5"]))
    return int((done / total) * 100)


def job_metrics(job: Dict[str, Any]) -> Dict[str, float]:
    comp = job.get("components") or {}
    return {
        "match": float(job.get("match_percent") or 0.0),
        "skill": float(comp.get("skill_overlap") or 0.0) * 100,
        "exp": float(comp.get("experience_alignment") or 0.0) * 100,
        "ats": float(comp.get("ats_score") or 0.0) * 100,
    }


def main() -> None:
    st.set_page_config(page_title="CareerAgent-AI — Glass-Box Mission Control", layout="wide")
    st.title("CareerAgent-AI — Glass-Box Mission Control")

    if "run_id" not in st.session_state:
        st.session_state["run_id"] = ""
    if "selected_urls" not in st.session_state:
        st.session_state["selected_urls"] = set()

    # Sidebar
    with st.sidebar:
        st.subheader("Autonomy Controls")
        default_th = st.slider("Default threshold", 0.0, 1.0, 0.70, 0.01)
        parser_th = st.slider("Parser threshold", 0.0, 1.0, 0.70, 0.01)
        discovery_th = st.slider("Discovery threshold", 0.0, 1.0, 0.70, 0.01)
        match_th = st.slider("Match threshold", 0.0, 1.0, 0.70, 0.01)
        max_ref = st.slider("Max retries / refinements", 0, 5, 3, 1)

        roles = st.text_input("Target roles (comma-separated)", "Data Scientist, ML Engineer, GenAI Engineer")
        location = st.text_input("Location", "United States")
        country = st.selectbox("Country", ["US", "CA", "UK", "IN", "AU"], index=0)
        remote = st.checkbox("Remote preferred", value=True)
        wfo_ok = st.checkbox("On-site/WFO acceptable", value=True)
        visa = st.checkbox("Visa sponsorship required (F1/OPT/H1B)", value=True)
        recency = st.number_input("Recency hours", min_value=6, max_value=168, value=36, step=6)
        max_jobs = st.number_input("Max jobs to hunt", min_value=10, max_value=60, value=40, step=5)
        phone = st.text_input("Phone for SMS (optional)", "")

        st.markdown("---")
        st.subheader("Upload")
        up = st.file_uploader("Resume (PDF/TXT/DOCX)", type=["pdf", "txt", "docx"])

        if st.button("🚀 Start Hunt", use_container_width=True, disabled=(up is None)):
            prefs = {
                "target_roles": [r.strip() for r in roles.split(",") if r.strip()][:4],
                "country": country,
                "location": location,
                "remote": remote,
                "wfo_ok": wfo_ok,
                "salary": "",
                "visa_sponsorship_required": visa,
                "recency_hours": float(recency),
                "max_jobs": int(max_jobs),
                "max_refinements": int(max_ref),
                "user_phone": phone or None,
                "thresholds": {
                    "default": float(default_th),
                    "parser": float(parser_th),
                    "discovery": float(discovery_th),
                    "match": float(match_th),
                    "draft": float(default_th),
                },
            }
            files = {"resume": (up.name, up.getvalue(), up.type or "application/octet-stream")}
            data = {"preferences_json": json.dumps(prefs)}
            r = requests.post(api_base() + "/analyze", files=files, data=data, timeout=120)
            r.raise_for_status()
            st.session_state["run_id"] = r.json()["run_id"]
            st.session_state["selected_urls"] = set()
            st.success(f"Run started: {st.session_state['run_id']}")

        st.markdown("---")
        st.subheader("Existing Run")
        st.session_state["run_id"] = st.text_input("Run ID", st.session_state["run_id"])

    run_id = st.session_state["run_id"].strip()
    if not run_id:
        st.info("Upload a resume and click Start Hunt.")
        return

    state = api_get(f"/status/{run_id}")

    # Top row
    c1, c2, c3, c4 = st.columns([2, 2, 2, 2])
    c1.markdown(f"**Run:** `{run_id}`")
    c2.markdown(f"**Status:** `{state.get('status')}`")
    c3.markdown(f"**Pending:** `{state.get('pending_action')}`")
    prog = calc_progress(state)
    c4.markdown(f"**Progress:** `{prog}%`")
    st.progress(prog / 100.0)

    # Feed
    st.subheader("Live Agent Feed")
    feed = state.get("live_feed") or []
    for e in feed[-20:]:
        st.write(f"[{e.get('layer')} {e.get('agent')}] {e.get('message')}")

    tabs = st.tabs(["Approval Grid", "Draft Review", "Engineer", "Artifacts"])

    # --- Approval Grid
    with tabs[0]:
        st.markdown("### Human Approval — Ranking")
        pending = str(state.get("pending_action") or "")
        st.caption("When pending_action = review_ranking, select jobs and approve to generate drafts.")

        ranking = state.get("ranking") or []
        if not ranking:
            ranking = load_json_artifact((state.get("artifacts") or {}).get("ranking")) or []

        if not ranking:
            st.warning("Ranking not available yet.")
        else:
            # preselect top 10 once
            if not st.session_state["selected_urls"]:
                for j in ranking[:10]:
                    u = str(j.get("url") or j.get("job_id") or "")
                    if u:
                        st.session_state["selected_urls"].add(u)

            for idx, job in enumerate(ranking[:30], start=1):
                url = str(job.get("url") or job.get("job_id") or "")
                title = job.get("title") or "(untitled)"
                m = job_metrics(job)

                with st.expander(f"{idx}. {title} — {m['match']:.1f}%"):
                    cols = st.columns([2.2, 1, 1, 1, 1.2])
                    cols[0].markdown(f"[Open job link]({url})")
                    cols[1].metric("Skill", f"{m['skill']:.0f}%")
                    cols[2].metric("Exp", f"{m['exp']:.0f}%")
                    cols[3].metric("ATS", f"{m['ats']:.0f}%")
                    sel = cols[4].checkbox("Select", value=(url in st.session_state["selected_urls"]), key=f"sel_{idx}")
                    if sel and url:
                        st.session_state["selected_urls"].add(url)
                    if (not sel) and url and url in st.session_state["selected_urls"]:
                        st.session_state["selected_urls"].remove(url)

                    if job.get("matched_skills"):
                        st.caption("Matched skills: " + ", ".join(job["matched_skills"][:15]))
                    if job.get("missing_skills"):
                        st.caption("Missing skills: " + ", ".join(job["missing_skills"][:12]))

            colA, colB = st.columns([1, 1])
            with colA:
                if st.button("✅ Approve Selected → Generate Drafts", use_container_width=True, disabled=(len(st.session_state["selected_urls"]) == 0)):
                    api_post(f"/action/{run_id}", {"action_type": "approve_ranking", "payload": {"selected_job_urls": sorted(list(st.session_state["selected_urls"]))}})
                    st.success("Approved ranking. Go to Draft Review tab and refresh.")
            with colB:
                reason = st.text_input("Reject reason (optional)", "")
                if st.button("❌ Reject Ranking → Re-run Hunt", use_container_width=True):
                    api_post(f"/action/{run_id}", {"action_type": "reject_ranking", "payload": {"reason": reason}})
                    st.warning("Ranking rejected. Pipeline restarted.")

    # --- Draft Review
    with tabs[1]:
        st.markdown("### Human Approval — Draft Review")
        artifacts = state.get("artifacts") or {}
        resume_keys = sorted([k for k in artifacts.keys() if k.startswith("resume_")])
        cover_keys = sorted([k for k in artifacts.keys() if k.startswith("cover_")])

        if not resume_keys and not cover_keys:
            st.info("No drafts found yet. Approve ranking first.")
        else:
            pick = st.selectbox("Select a draft artifact", resume_keys + cover_keys)
            if pick:
                p = Path(artifacts[pick]["path"])
                text = p.read_text(encoding="utf-8") if p.exists() else "(file missing)"
                st.code(text[:12000], language="markdown")
                st.download_button("Download file", data=text.encode("utf-8"), file_name=p.name, mime="text/markdown")

            col1, col2 = st.columns([1, 1])
            with col1:
                if st.button("✅ Approve Drafts → Finalize (L7–L9)", use_container_width=True):
                    api_post(f"/action/{run_id}", {"action_type": "approve_drafts", "payload": {}})
                    st.success("Approved drafts. Refresh to see completion.")
            with col2:
                r = st.text_input("Draft reject reason", "")
                if st.button("❌ Reject Drafts → Back to Ranking", use_container_width=True):
                    api_post(f"/action/{run_id}", {"action_type": "reject_drafts", "payload": {"reason": r}})
                    st.warning("Drafts rejected. Back to ranking.")

    # --- Engineer
    with tabs[2]:
        st.markdown("### Engineer View — Run a Layer")
        cols = st.columns(5)
        for i, layer in enumerate(["L0", "L2", "L3", "L4", "L5"]):
            if cols[i].button(f"Run {layer}", use_container_width=True):
                api_post(f"/action/{run_id}", {"action_type": "execute_layer", "payload": {"layer": layer}})
                st.success(f"Executed {layer}. Refresh.")

        st.markdown("### Tool Audit (last 20 attempts)")
        for a in (state.get("attempts") or [])[-20:]:
            st.write(f"- [{a.get('layer_id')}] {a.get('tool')} → {a.get('status')} (conf={a.get('confidence')})")

    # --- Artifacts
    with tabs[3]:
        st.markdown("### Artifacts (local-first)")
        artifacts = state.get("artifacts") or {}
        for k, v in artifacts.items():
            st.write(f"- **{k}** → `{v.get('path','')}`")

Overwriting app/ui/dashboard.py


In [ ]:
# # app/main.py
# from __future__ import annotations

# import sys
# from pathlib import Path

# ROOT = Path(__file__).resolve().parents[1]
# if str(ROOT) not in sys.path:
#     sys.path.insert(0, str(ROOT))

# SRC = ROOT / "src"
# if str(SRC) not in sys.path:
#     sys.path.insert(0, str(SRC))

# from app.ui.mission_control import main

# main()

In [7]:
%%writefile app/main.py
from __future__ import annotations

from app.ui.dashboard import main

if __name__ == "__main__":
    main()

Overwriting app/main.py
